In [0]:
import os
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow.keras.datasets.mnist import load_data
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.layers import Input, Conv2D, Add, BatchNormalization, concatenate

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score

In [2]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)
os.chdir('/gdrive/My Drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
newPath='./데이콘 정리된데이터'
submission = pd.read_csv(newPath+'/sample_submission.csv')

In [0]:
test_refined = np.load(newPath+"/test_refined.npy")
train_refined = np.load(newPath+"/train_refined-002.npy")

In [5]:
print(test_refined.shape,train_refined.shape)

(2416, 40, 40, 14) (75917, 40, 40, 15)


In [6]:
num=[]
n = 50
for i in range(train_refined.shape[0]):
    if ((train_refined[i,:,:,-1]>0).sum() > n) and ((train_refined[i,:,:,-1]<0).sum() <= 0):
        num.append(i)

print("사용하는 데이터의 양:",len(num) / train_refined.shape[0])
Reduced_xtrain=train_refined[num,:,:,:]
del train_refined

사용하는 데이터의 양: 0.4011222782775926


In [0]:
Xtrain = Reduced_xtrain[:, :, :, :9] #<- 같은 이름으로 저장하면 다음 코드에서 저장안됨
Ytrain = Reduced_xtrain[:, :, :,14]

test_refined = test_refined[:, :, :, :9] #<- 9개의 feature만, 지형 정보사용 X

del Reduced_xtrain #<- RAM 조절하기 윟해 삭제

In [8]:
print("Xtrain.shape:",Xtrain.shape)
print("Ytrain.shape:",Ytrain.shape) 


Xtrain.shape: (30452, 40, 40, 9)
Ytrain.shape: (30452, 40, 40)


In [9]:
tempXtrain = Xtrain.reshape(-1,9)
tempXtrain.shape

(48723200, 9)

In [0]:
del Xtrain

In [11]:
# 2,3,4,5,6 채널의 차이를 구하여 합함
from itertools import combinations
for i, j in list(combinations(range(5), 2)):
   tempXtrain = np.hstack((tempXtrain, (tempXtrain[:, j+2] - tempXtrain[:, i+2]).reshape(-1, 1)))
tempXtrain.shape

(48723200, 19)

In [0]:
Xtrain=tempXtrain.reshape(30452,40,40,19)
del tempXtrain

#4. 테스트용 train_test_split 나누기

In [13]:
xtrain,xtest,ytrain,ytest = train_test_split(Xtrain,Ytrain, test_size=0.025)
del Xtrain
del Ytrain #<- RAM 사용량 때문에 지움
print("xtrain.shape:",xtrain.shape,"xtest.shape:",xtest.shape,"\n"
      "ytrain.shape:",ytrain.shape,"ytest.shape:",ytest.shape)

xtrain.shape: (29690, 40, 40, 19) xtest.shape: (762, 40, 40, 19) 
ytrain.shape: (29690, 40, 40) ytest.shape: (762, 40, 40)


In [0]:
def mae_over_fscore(y_true, y_pred):
    '''
    y_true: sample_submission.csv 형태의 실제 값
    y_pred: sample_submission.csv 형태의 예측 값
    '''

    y_true = np.array(y_true)
    y_true = y_true.reshape(1, -1)[0]  
    
    y_pred = np.array(y_pred)
    y_pred = y_pred.reshape(1, -1)[0]
    
    # 실제값이 0.1 이상인 픽셀의 위치 확인
    IsGreaterThanEqualTo_PointOne = y_true >= 0.1
    
    # 실제 값에 결측값이 없는 픽셀의 위치 확인 
    IsNotMissing = y_true >= 0
    
    # mae 계산
    mae = np.mean(np.abs(y_true[IsGreaterThanEqualTo_PointOne] - y_pred[IsGreaterThanEqualTo_PointOne]))
    
    # f1_score 계산 위해, 실제값에 결측값이 없는 픽셀에 대해 1과 0으로 값 변환
    y_true = np.where(y_true[IsNotMissing] >= 0.1, 1, 0)
    
    y_pred = np.where(y_pred[IsNotMissing] >= 0.1, 1, 0)
    
    # f1_score 계산    
    f_score = f1_score(y_true, y_pred) 
    # f1_score가 0일 나올 경우를 대비하여 소량의 값 (1e-07) 추가 
    return mae / (f_score + 1e-07) 

In [0]:
def mae(y_true, y_pred):    
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    
    y_true = y_true.reshape(1, -1)[0]
    
    y_pred = y_pred.reshape(1, -1)[0]
    
    over_threshold = y_true >= 0.1
    
    return np.mean(np.abs(y_true[over_threshold] - y_pred[over_threshold]))

def fscore(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    
    y_true = y_true.reshape(1, -1)[0]
    
    y_pred = y_pred.reshape(1, -1)[0]
    
    remove_NAs = y_true >= 0
    
    y_true = np.where(y_true[remove_NAs] >= 0.1, 1, 0)
    
    y_pred = np.where(y_pred[remove_NAs] >= 0.1, 1, 0)
    
    return(f1_score(y_true, y_pred))

def maeOverFscore(y_true, y_pred):
    return mae(y_true, y_pred) / (fscore(y_true, y_pred) + 1e-07)

def fscore_keras(y_true, y_pred):
    score = tf.py_function(func=fscore, inp=[y_true, y_pred], Tout=tf.float32, name='fscore_keras')
    return score

def score(y_true, y_pred):
    score = tf.py_function(func=maeOverFscore, inp=[y_true, y_pred], Tout=tf.float32,  name='custom_mse') 
    return score

In [0]:
def create_model():
    inputs=Input(xtrain.shape[1:])
    
    bn=BatchNormalization()(inputs)
    conv0=Conv2D(256, kernel_size=1, strides=1, padding='same', activation='relu')(bn)
    
    bn=BatchNormalization()(conv0)
    conv=Conv2D(128, kernel_size=2, strides=1, padding='same', activation='relu')(bn)
    concat=concatenate([conv0, conv], axis=3)
    
    bn=BatchNormalization()(concat)
    conv=Conv2D(64, kernel_size=3, strides=1, padding='same', activation='relu')(bn)
    concat=concatenate([concat, conv], axis=3)
        
    for i in range(5):
        bn=BatchNormalization()(concat)
        conv=Conv2D(32, kernel_size=3, strides=1, padding='same', activation='relu')(bn)
        concat=concatenate([concat, conv], axis=3)
    
    bn=BatchNormalization()(concat)
    outputs=Conv2D(1, kernel_size=1, strides=1, padding='same', activation='relu')(bn)
    
    model=Model(inputs=inputs, outputs=outputs)
    
    return model

In [0]:
Model = create_model() 

In [0]:
Model.compile(loss='mae', optimizer='adam', metrics=[score,fscore_keras])

In [19]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

ReduceLROnPlateau = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',patience=7) # factor=0.05
EarlyStop = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=7,restore_best_weights=True)

Using TensorFlow backend.


In [0]:
callback=[EarlyStop, ReduceLROnPlateau]

In [21]:
hist = Model.fit(xtrain,ytrain,batch_size=64, epochs=10, verbose=1, callbacks=callback)

Epoch 1/10
464/464 [==============================] - 264s 570ms/step - loss: 0.2731 - score: 2.3826 - fscore_keras: 0.6808 - lr: 0.0010
Epoch 2/10
464/464 [==============================] - 260s 561ms/step - loss: 0.2322 - score: 1.9083 - fscore_keras: 0.7426 - lr: 0.0010
Epoch 3/10
464/464 [==============================] - 260s 561ms/step - loss: 0.2280 - score: 1.8443 - fscore_keras: 0.7523 - lr: 0.0010
Epoch 4/10
464/464 [==============================] - 261s 562ms/step - loss: 0.2247 - score: 1.7977 - fscore_keras: 0.7613 - lr: 0.0010
Epoch 5/10
464/464 [==============================] - 260s 560ms/step - loss: 0.2226 - score: 1.7680 - fscore_keras: 0.7675 - lr: 0.0010
Epoch 6/10
464/464 [==============================] - 260s 560ms/step - loss: 0.2211 - score: 1.7471 - fscore_keras: 0.7703 - lr: 0.0010
Epoch 7/10
464/464 [==============================] - 260s 560ms/step - loss: 0.2198 - score: 1.7357 - fscore_keras: 0.7724 - lr: 0.0010
Epoch 8/10
464/464 [=====================

In [22]:
evaluation = Model.evaluate(xtest,ytest)

24/24 [==============================] - 4s 148ms/step - loss: 0.2431 - score: 1.8668 - fscore_keras: 0.7760


In [23]:
print("EarlyStop,ReduceLROnPlateau,accuracy:",'\n',
      evaluation)

EarlyStop,ReduceLROnPlateau,accuracy: 
 [0.24311287701129913, 1.8668127059936523, 0.7759681344032288]


In [0]:
Model.save("./모델저장폴더/05-25 Resnet using refined DATA E10 B64.h5")